In [57]:
import pandas as pd
import re

In [58]:
df = pd.read_csv("forms_responses.csv")
df.head()

,Carimbo de data/hora,Qual a sua idade?,Você faz uso regular de um ou mais dos medicamentos listados abaixos?,Você pratica algum tipo de exercício físico regularmente?,"Normalmente, quantas vezes por semana você pratica exercícios físicos?",Os exercícios que você mais pratica envolvem esforço cardiovascular intenso?,Quais exercícios físicos você mais pratica?,"Em média, suas sessões de exercício físico duram quantos minutos?",Em que período do dia você é mais acostumado a realizar a prática de exercícios?,Quantas horas por noite você costuma dormir?,"Como você classificaria a qualidade do seu sono num geral, de 1 a 5?","Na sua visão, a prática de exercícios físicos afeta de alguma forma a qualidade do seu sono?","Se quiser, desenvolva como você acredita que a prática de exercícios afeta a qualidade do seu sono.",Quantas vezes você praticou exercícios nos últimos 7 dias?,"Como você classificaria a qualidade do seu sono dos últimos 7 dias, de 1 a 5?","Nos últimos 7 dias, algo externo (que não seja a prática de esportes) pode ter afetado seu sono?"
0,28/04/2025 21:09:39,21,Não faço uso de nenhum medicamento controlado ...,Não,1,Não,Caminhada,15,Manhã,7,3,"Sim, afeta para melhor",NaN,1,3,"Sim, algo afetou meu sono para pior"
1,28/04/2025 21:16:31,22,"Ansiolíticos (alprazolam, clonazepam, diazepam...",Sim,4,Sim,"Caminhada, Corrida, Treino de força (musculaçã...",90,Tarde,6,2,"Sim, afeta para melhor",NaN,0,3,"Sim, algo afetou meu sono para melhor"
2,28/04/2025 21:16:32,23,Não faço uso de nenhum medicamento controlado ...,Não,0,Não,Caminhada,10,Manhã,9,3,"Não, não afeta",NaN,5 ou mais,3,Não tenho certeza
3,28/04/2025 21:18:08,21,Não faço uso de nenhum medicamento controlado ...,Sim,4,Sim,Treino de força (musculação),120,Noite,6,4,"Sim, afeta para melhor",NaN,2,4,Não
4,28/04/2025 21:18:13,21,Não faço uso de nenhum medicamento controlado ...,Não,0,Não,Caminhada,0,Tarde,6,3,"Não, não afeta",NaN,1,1,"Sim, algo afetou meu sono para pior"


**É possível observar algumas coisas a serem organizadas:** 
* As colunas recebem o nome da pergunta -> nomes muito longos
* Algumas respostas (não validadas no formulário) estão sem padrão. Por exemplo, na coluna "idade", temos tanto respostas numéricas quanto strings ("20 anos").
* Temos (pouquíssimas) respostas com erro de preenchimento na pergunta sobre a duração das sessões de exercícios
* As perguntas com múltiplas respostas precisam ser organizadas para serem analisadas

**Renomear as colunas**

In [59]:
df.columns

Index(['Carimbo de data/hora', 'Qual a sua idade?',
       'Você faz uso regular de um ou mais dos medicamentos listados abaixos?',
       'Você pratica algum tipo de exercício físico regularmente?',
       'Normalmente, quantas vezes por semana você pratica exercícios físicos?',
       'Os exercícios que você mais pratica envolvem esforço cardiovascular intenso?',
       'Quais exercícios físicos você mais pratica?',
       'Em média, suas sessões de exercício físico duram quantos minutos?',
       'Em que período do dia você é mais acostumado a realizar a prática de exercícios?',
       'Quantas horas por noite você costuma dormir?',
       'Como você classificaria a qualidade do seu sono num geral, de 1 a 5?',
       'Na sua visão, a prática de exercícios físicos afeta de alguma forma a qualidade do seu sono?',
       'Se quiser, desenvolva como você acredita que a prática de exercícios afeta a qualidade do seu sono.',
       'Quantas vezes você praticou exercícios nos últimos 7 dia

In [60]:
df.rename(columns={
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade?': 'age',
    'Você faz uso regular de um ou mais dos medicamentos listados abaixos?': 'medication',
    'Você pratica algum tipo de exercício físico regularmente?': 'exercise_practice',
    'Normalmente, quantas vezes por semana você pratica exercícios físicos?': 'exercise_freq',
    'Os exercícios que você mais pratica envolvem esforço cardiovascular intenso?': 'exercise_cardio',
    'Quais exercícios físicos você mais pratica?': 'exercise_types',
    'Em média, suas sessões de exercício físico duram quantos minutos?': 'exercise_duration',
    'Em que período do dia você é mais acostumado a realizar a prática de exercícios?': 'exercise_period',
    'Quantas horas por noite você costuma dormir?': 'sleep_duration',
    'Como você classificaria a qualidade do seu sono num geral, de 1 a 5?': 'sleep_quality',
    'Na sua visão, a prática de exercícios físicos afeta de alguma forma a qualidade do seu sono?': 'sleep_affects',
    'Se quiser, desenvolva como você acredita que a prática de exercícios afeta a qualidade do seu sono.' : 'opinion',
    'Quantas vezes você praticou exercícios nos últimos 7 dias?': 'recent_exercise_frequency',
    'Como você classificaria a qualidade do seu sono dos últimos 7 dias, de 1 a 5?': 'recent_sleep_quality',
    'Nos últimos 7 dias, algo externo (que não seja a prática de esportes) pode ter afetado seu sono?': 'recent_ext_factors'
}, inplace = True)
df.head()

,timestamp,age,medication,exercise_practice,exercise_freq,exercise_cardio,exercise_types,exercise_duration,exercise_period,sleep_duration,sleep_quality,sleep_affects,opinion,recent_exercise_frequency,recent_sleep_quality,recent_ext_factors
0,28/04/2025 21:09:39,21,Não faço uso de nenhum medicamento controlado ...,Não,1,Não,Caminhada,15,Manhã,7,3,"Sim, afeta para melhor",NaN,1,3,"Sim, algo afetou meu sono para pior"
1,28/04/2025 21:16:31,22,"Ansiolíticos (alprazolam, clonazepam, diazepam...",Sim,4,Sim,"Caminhada, Corrida, Treino de força (musculaçã...",90,Tarde,6,2,"Sim, afeta para melhor",NaN,0,3,"Sim, algo afetou meu sono para melhor"
2,28/04/2025 21:16:32,23,Não faço uso de nenhum medicamento controlado ...,Não,0,Não,Caminhada,10,Manhã,9,3,"Não, não afeta",NaN,5 ou mais,3,Não tenho certeza
3,28/04/2025 21:18:08,21,Não faço uso de nenhum medicamento controlado ...,Sim,4,Sim,Treino de força (musculação),120,Noite,6,4,"Sim, afeta para melhor",NaN,2,4,Não
4,28/04/2025 21:18:13,21,Não faço uso de nenhum medicamento controlado ...,Não,0,Não,Caminhada,0,Tarde,6,3,"Não, não afeta",NaN,1,1,"Sim, algo afetou meu sono para pior"


**Padronizar os dados**

In [61]:

df["age"] = df["age"].apply(lambda x: int(x.lower().replace("anos", "")))
df["exercise_freq"]= df["exercise_freq"].apply(lambda x: int(x.replace(" ou mais", "")))
df["recent_exercise_frequency"]= df["recent_exercise_frequency"].apply(lambda x: int(x.replace(" ou mais", "")))

df["exercise_types"] = df["exercise_types"].apply(lambda x: re.sub(r"\([^)]*\)", "", x.lower()))

df["medication"] = df["medication"].apply(lambda x: re.sub(r"\([^)]*\)", "", x.lower()))
df["medication"] = df["medication"].apply(lambda x: x.replace("não faço uso de nenhum medicamento controlado que afete o sono.", "nenhum"))
df["medication"] = df["medication"].apply(lambda x: x.replace("outro medicamento controlado que possa afetar o sono", "outro"))

df["recent_ext_factors"] = df["recent_ext_factors"].apply(lambda x: x.lower())
df["recent_ext_factors"] = df["recent_ext_factors"].apply(lambda x: x.replace("sim, algo afetou meu sono para melhor", "melhorou"))
df["recent_ext_factors"] = df["recent_ext_factors"].apply(lambda x: x.replace("não tenho certeza", "unsure"))
df["recent_ext_factors"] = df["recent_ext_factors"].apply(lambda x: x.replace("sim, algo afetou meu sono para pior", "piorou"))

**Remover respostas com erro de preenchimento**

In [62]:
df.shape

(39, 16)

In [63]:
# sessões que duram 0 minutos ou mais que 5
df = df[(df["exercise_duration"] == 0) | (df["exercise_duration"] > 5)]
df.shape

(37, 16)

**Organizando as perguntas de múltiplas respostas**  
Temos duas colunas com vários valores separados por vírgula: medicação e tipo de exercício físico. Vamos transformar as respostas possíveis em colunas binárias.

In [64]:
# tratamento - coluna medicação

medication_dummies = df["medication"].str.get_dummies(sep=" , ")
medication_dummies.head()

,ansiolíticos,antidepressivos,antidepressivos,estimulantes,indutores de sono,nenhum,outro
0,0,0,0,0,0,1,0
1,1,1,0,0,1,0,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0


In [65]:
# verifica se tem alguma inconsistência nas respostas
s = medication_dummies.drop(columns=["nenhum"]).sum(axis=1)
e = medication_dummies[(medication_dummies["nenhum"] == 1) & (s > 0)]
print(e)

Empty DataFrame
Columns: [ansiolíticos, antidepressivos, antidepressivos , estimulantes , indutores de sono , nenhum, outro ]
Index: []


In [66]:
# tratamento - coluna tipo de exercício

exercise_types_dummies = df["exercise_types"].str.get_dummies(sep=", ")
exercise_types_dummies.head()

,caminhada,ciclismo,corrida,esgrima,esportes coletivos,luta,natação,pilates,treino de força,yoga
0,1,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,1,1,1
2,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0


In [67]:
df = pd.concat([df, medication_dummies], axis=1)
df = pd.concat([df, exercise_types_dummies], axis=1)

In [68]:
df.shape

(37, 33)

In [69]:
# remover colunas originais de medicamento e exercícios
df.drop(columns=["medication", "exercise_types"], inplace=True)
df.shape

(37, 31)

**Agora, basta somente exportar esse dataset para um novo csv, para fazermos análises em seguida.**

In [70]:
df.to_csv("forms_responses_clear.csv", index = False)